In [ ]:
from numpy import max
import pygmsh, matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
with pygmsh.geo.Geometry() as geom:
        #geom.add_rectangle(0.0, 1.0, 0.0, 1.0, 0.0, 0.05)
        geom.add_circle([0.0, 0.0], 1.0, mesh_size=0.1)
        msh = geom.generate_mesh()
coord=msh.points
node = msh.cells_dict["triangle"]
line = msh.cells_dict["line"]
vertex = msh.cells_dict["vertex"]
coord = np.delete(coord, (0), axis=0)#Elimina primera fila de la matriz coordenadas
node =node-1
line=line-1
#display(node)
nnode=len(coord)#Numero de nodos
ntri=len(node)#Numero de triangulos
fig,axs=plt.subplots()
x=coord[:,0]
y=coord[:,1]
axs.triplot(x,y,node)
axs.set_aspect('equal')
dirichlet = np.unique(line)
x_dirichlet=x[dirichlet]
y_dirichlet=y[dirichlet]
#h=axs.plot(x_dirichlet,y_dirichlet,'bo')
#Agrega numero de nodos
for nodo in range(nnode):
    plt.text(x[nodo],y[nodo],str(nodo))


# Funciones de forma locales

<img src="bases.jpg" style="width: 60%"/>

In [ ]:
#Calculo de funciones de forma locales
from sympy import Matrix,symbols
x1,y1,x2,y2,x3,y3=symbols('x1,y1,x2,y2,x3,y3')
mm1=Matrix([[1,x1,y1],[1,x2,y2],[1,x3,y3]])
mm=mm1.inv()
deter=mm1.det()
print(deter)

In [ ]:
def pdetrg(xx,yy):
    x1=xx[:,0]
    x2=xx[:,1]
    x3=xx[:,2]
    y1=yy[:,0]
    y2=yy[:,1]
    y3=yy[:,2]
    gx=np.concatenate(([(y2 - y3)/(x1*y2 - x1*y3 - x2*y1 + x2*y3 + x3*y1 - x3*y2)],[(-y1 + y3)/(x1*y2 - x1*y3 - x2*y1 + x2*y3 + x3*y1 - x3*y2)],[(y1 - y2)/(x1*y2 - x1*y3 - x2*y1 + x2*y3 + x3*y1 - x3*y2)]),axis=0)
    gy=np.concatenate(([(-x2 + x3)/(x1*y2 - x1*y3 - x2*y1 + x2*y3 + x3*y1 - x3*y2)],[(x1 - x3)/(x1*y2 - x1*y3 - x2*y1 + x2*y3 + x3*y1 - x3*y2)],[(-x1 + x2)/(x1*y2 - x1*y3 - x2*y1 + x2*y3 + x3*y1 - x3*y2)]),axis=0)
    gx=np.transpose(gx)
    gy=np.transpose(gy)
    A=(x1*y2 - x1*y3 - x2*y1 + x2*y3 + x3*y1 - x3*y2)/2
    return A, gx,gy

In [ ]:
xx=x[node]
yy=y[node]


In [ ]:
A,gx,gy=pdetrg(xx,yy)
print(gy)

In [ ]:
from scipy.sparse import csr_matrix,spmatrix,dok_matrix
from scipy.sparse.linalg import spsolve

In [ ]:
K=dok_matrix((nnode, nnode), dtype='float')
F=dok_matrix((nnode, 1), dtype='float')
#Ensambla matriz K
for i in range(3):
    for j in range(3):
        K+=csr_matrix((A*(gx[:,i]*gx[:,j]+gy[:,i]*gy[:,j]),(node[:,i],node[:,j])),shape=(nnode,nnode),dtype='float')
#Ensambla vector de fuerzas
for i in range(3):
        F+=csr_matrix((-A/3,(node[:,i],np.zeros(ntri))),shape=(nnode,1),dtype='float')

In [ ]:
gdlR=dirichlet
gdlL=np.setdiff1d(range(nnode),gdlR)

auxones=np.ones(len(gdlL))
#display(auxones)
mataux=csr_matrix((auxones,(gdlL,range(len(gdlL)))),shape=(nnode,len(gdlL)),dtype='float')
KLL=mataux.T*K*mataux
FL=mataux.T*F
u=np.zeros(nnode,dtype='float')
uL=spsolve(KLL,FL)


u[gdlL]=uL


In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(1, 2, 1, projection='3d')
aux=ax1.plot_trisurf(x, y, u, triangles=node,cmap=plt.cm.Spectral)
uex=(x**2+y**2)/4-1/4
ax2 = fig.add_subplot(1,2, 2, projection='3d')
aux=ax2.plot_trisurf(x, y, uex, triangles=node,cmap=plt.cm.Spectral)


In [ ]:
np.min(u)/np.min(uex)